<h1><center>Laboratorio 9: Optimización de modelos 💯</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos - Primavera 2024</strong></center>

### **Cuerpo Docente:**

- Profesores: Ignacio Meza, Sebastián Tinoco
- Auxiliar: Eduardo Moya
- Ayudantes: Nicolás Ojeda, Melanie Peña, Valentina Rojas

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1:
- Nombre de alumno 2:


### **Link de repositorio de GitHub:** [Insertar Repositorio](https://github.com/...../)

### Temas a tratar

- Predicción de demanda usando `xgboost`
- Búsqueda del modelo óptimo de clasificación usando `optuna`
- Uso de pipelines.

### Reglas:

- **Grupos de 2 personas**
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias.
- Pueden usar cualquer matrial del curso que estimen conveniente.
- Código que no se pueda ejecutar, no será revisado.

### Objetivos principales del laboratorio

- Optimizar modelos usando `optuna`
- Recurrir a técnicas de *prunning*
- Forzar el aprendizaje de relaciones entre variables mediante *constraints*
- Fijar un pipeline con un modelo base que luego se irá optimizando.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `pandas`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre DataFrames.

# Importamos librerias útiles

In [1]:
!pip install -qq xgboost optuna

# El emprendimiento de Fiu

Tras liderar de manera exitosa la implementación de un proyecto de ciencia de datos para caracterizar los datos generados en Santiago 2023, el misterioso corpóreo **Fiu** se anima y decide levantar su propio negocio de consultoría en machine learning. Tras varias e intensas negociaciones, Fiu logra encontrar su *primera chamba*: predecir la demanda (cantidad de venta) de una famosa productora de bebidas de calibre mundial. Al ver el gran potencial y talento que usted ha demostrado en el campo de la ciencia de datos, Fiu lo contrata como data scientist para que forme parte de su nuevo emprendimiento.

Para este laboratorio deben trabajar con los datos `sales.csv` subidos a u-cursos, el cual contiene una muestra de ventas de la empresa para diferentes productos en un determinado tiempo.

Para comenzar, cargue el dataset señalado y visualice a través de un `.head` los atributos que posee el dataset.

<i><p align="center">Fiu siendo felicitado por su excelente desempeño en el proyecto de caracterización de datos</p></i>
<p align="center">
  <img src="https://media-front.elmostrador.cl/2023/09/A_UNO_1506411_2440e.jpg">
</p>

In [217]:
import pandas as pd
import numpy as np
from datetime import datetime

df = pd.read_csv('sales.csv')

df.head()

,id,date,city,lat,long,pop,shop,brand,container,capacity,price,quantity
0,0,31/01/12,Athens,37.97945,23.71622,672130,shop_1,kinder-cola,glass,500ml,0.96,13280
1,1,31/01/12,Athens,37.97945,23.71622,672130,shop_1,kinder-cola,plastic,1.5lt,2.86,6727
2,2,31/01/12,Athens,37.97945,23.71622,672130,shop_1,kinder-cola,can,330ml,0.87,9848
3,3,31/01/12,Athens,37.97945,23.71622,672130,shop_1,adult-cola,glass,500ml,1.00,20050
4,4,31/01/12,Athens,37.97945,23.71622,672130,shop_1,adult-cola,can,330ml,0.39,25696


In [203]:
# Cantidad de valores únicos por columna
df.nunique()

id           7456
date           84
city            5
lat             6
long            6
pop            35
shop            6
brand           5
container       3
capacity        3
price         402
quantity     6906
dtype: int64

## 1 Generando un Baseline (5 puntos)

<p align="center">
  <img src="https://media.tenor.com/O-lan6TkadUAAAAC/what-i-wnna-do-after-a-baseline.gif">
</p>

Antes de entrenar un algoritmo, usted recuerda los apuntes de su magíster en ciencia de datos y recuerda que debe seguir una serie de *buenas prácticas* para entrenar correcta y debidamente su modelo. Después de un par de vueltas, llega a las siguientes tareas:

1. Separe los datos en conjuntos de train (70%), validation (20%) y test (10%). Fije una semilla para controlar la aleatoriedad. [0.5 puntos]
2. Implemente un `FunctionTransformer` para extraer el día, mes y año de la variable `date`. Guarde estas variables en el formato categorical de pandas. [1 punto]
3. Implemente un `ColumnTransformer` para procesar de manera adecuada los datos numéricos y categóricos. Use `OneHotEncoder` para las variables categóricas. `Nota:` Utilice el método `.set_output(transform='pandas')` para obtener un DataFrame como salida del `ColumnTransformer` [1 punto]
4. Guarde los pasos anteriores en un `Pipeline`, dejando como último paso el regresor `DummyRegressor` para generar predicciones en base a promedios. [0.5 punto]
5. Entrene el pipeline anterior y reporte la métrica `mean_absolute_error` sobre los datos de validación. ¿Cómo se interpreta esta métrica para el contexto del negocio? [0.5 puntos]
6. Finalmente, vuelva a entrenar el `Pipeline` pero esta vez usando `XGBRegressor` como modelo **utilizando los parámetros por default**. ¿Cómo cambia el MAE al implementar este algoritmo? ¿Es mejor o peor que el `DummyRegressor`? [1 punto]
7. Guarde ambos modelos en un archivo .pkl (uno cada uno) [0.5 puntos]

In [218]:
from sklearn import set_config
set_config(transform_output="pandas")

**1. Separación de conjuntos**

Dado que la variable que queremos predecir es `quantity`, primero separaremos el dataset entre *datos* y *labels*

In [219]:
from sklearn.model_selection import train_test_split

X = df.drop(columns = ['id', 'quantity'])
y = df['quantity']

X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.1, random_state=30)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.222, random_state=30)

**2. Implementación `FunctionTransformer`**

In [220]:
from sklearn.preprocessing import FunctionTransformer

def date_extractor(df):
    '''
    Extrae el día, mes y año de una la columna 'date'

    Args
        df: DataFrame

    Returns
        transdformed_df: DataFrame con las columnas 'day', 'month' y 'year' y eliminada 'date'
    '''
    # Creación de las columnas separadas
    transformed_df = df.copy()
    transformed_df['date'] = pd.to_datetime(transformed_df['date'], format='%d/%m/%y', dayfirst = True)
    transformed_df['day'] = transformed_df['date'].dt.day.astype('category')
    transformed_df['month'] = transformed_df['date'].dt.month.astype('category')
    transformed_df['year'] = transformed_df['date'].dt.year.astype('category')
    transformed_df = transformed_df.drop(columns = 'date')

    # Guardado

    return transformed_df

# Creamos el transformer para aplicar esta nueva función
date_transformer = FunctionTransformer(date_extractor)

**3. Implementación `ColumnTransformer`**

In [221]:
df.describe()

,id,lat,long,pop,price,quantity
count,7456.000000,7456.000000,7456.000000,7456.000000,7456.000000,7456.000000
mean,3784.926770,38.300616,23.270170,355042.733637,1.197193,29408.428380
std,2185.822361,1.650030,1.086592,232336.703020,0.818175,17652.985675
min,0.000000,35.327870,21.734440,134219.000000,0.110000,2953.000000
25%,1889.750000,37.962450,22.417610,141732.000000,0.620000,16572.750000
50%,3783.500000,38.244440,22.930860,257501.500000,0.930000,25294.500000
75%,5682.250000,39.636890,23.716220,665102.000000,1.510000,37699.000000
max,7559.000000,40.643610,25.143410,672130.000000,4.790000,145287.000000


Haciendo una revisión de las variables, podemos decir que tenemos las siguientes categorías de datos:

1. **Variables Numéricas**: `lat`, `long`, `pop`, `capacity`, `price`, `container`, `quantity`
2. **Variables Categóricas**: `city`, `shop`, `brand`, `day`, `month`, `year`

Además vamos a hacer un reemplazo dentro de la variable `container`. Dado que las únicas capacidades disponibles son `330ml`, `500ml` y `1.5lt`, podemos reemplazarlas por `300`, `500` y `1500` respectivamente, así mantenemos una misma unidad de medida.

In [222]:
replace = {
    '330ml': 330,
    '500ml': 500,
    '1.5lt': 1500,
}

X_train['capacity'] = X_train['capacity'].replace(replace)
X_val['capacity'] = X_val['capacity'].replace(replace)
X_test['capacity'] = X_test['capacity'].replace(replace)

In [223]:
from sklearn.preprocessing import MinMaxScaler , OneHotEncoder
from sklearn.compose import ColumnTransformer

num_columns = ['lat', 'long', 'pop', 'capacity', 'price']
cat_columns = ['city', 'shop', 'brand', 'container', 'day', 'month', 'year']

num_scaler = MinMaxScaler()
cat_encoder = OneHotEncoder(handle_unknown = 'infrequent_if_exist', sparse_output = False)

col_transformer = ColumnTransformer([
    ('num', num_scaler, num_columns),
    ('cat', cat_encoder, cat_columns)
])

col_transformer.set_output(transform='pandas')

ColumnTransformer(transformers=[('num', MinMaxScaler(),
                                 ['lat', 'long', 'pop', 'capacity', 'price']),
                                ('cat',
                                 OneHotEncoder(handle_unknown='infrequent_if_exist',
                                               sparse_output=False),
                                 ['city', 'shop', 'brand', 'container', 'day',
                                  'month', 'year'])])

**4. Creación de `Pipeline`**

In [224]:
from sklearn.pipeline import Pipeline
from sklearn.dummy import DummyRegressor

dummy_model = DummyRegressor()

dummy_pipe = Pipeline([
    ('date', date_transformer),
    ('col', col_transformer),
    ('model', dummy_model)
], verbose = True)

**5. Entrenamiento `Pipeline` con `DummyRegressor`**

In [225]:
dummy_pipe.fit(X_train, y_train)

train_pred = dummy_pipe.predict(X_train)
test_pred = dummy_pipe.predict(X_test)

[Pipeline] .............. (step 1 of 3) Processing date, total=   0.0s
[Pipeline] ............... (step 2 of 3) Processing col, total=   0.0s
[Pipeline] ............. (step 3 of 3) Processing model, total=   0.0s


In [226]:
from sklearn.metrics import mean_absolute_error

mae_train = mean_absolute_error(y_train, train_pred)
mae_test = mean_absolute_error(y_test, test_pred)

print(f'MAE Train: {mae_train:.2f}')
print(f'MAE Test: {mae_test:.2f}')

MAE Train: 13365.39
MAE Test: 14261.46


**6. Reentrenamiento `Pipeline` con `XGBRegressor`**

In [227]:
from xgboost import XGBRegressor

xgbr_model = XGBRegressor()

xgbr_pipe = Pipeline([
    ('date', date_transformer),
    ('col', col_transformer),
    ('model', xgbr_model)
], verbose = True)

In [228]:
xgbr_pipe.fit(X_train, y_train)

train_pred = xgbr_pipe.predict(X_train)
test_pred = xgbr_pipe.predict(X_test)

[Pipeline] .............. (step 1 of 3) Processing date, total=   0.0s


[Pipeline] ............... (step 2 of 3) Processing col, total=   0.0s
[Pipeline] ............. (step 3 of 3) Processing model, total=   0.7s


In [229]:
mae_train = mean_absolute_error(y_train, train_pred)
mae_test = mean_absolute_error(y_test, test_pred)

print(f'MAE Train: {mae_train:.2f}')
print(f'MAE Test: {mae_test:.2f}')

MAE Train: 1104.52
MAE Test: 2692.10


**7. Guardado de modelos**

In [230]:
# Guardamos ambos modelos en formato pickle
import joblib

joblib.dump(dummy_pipe, 'dummy_pipe.pkl')
joblib.dump(xgbr_pipe, 'xgbr_pipe.pkl')

['xgbr_pipe.pkl']

## 2. Forzando relaciones entre parámetros con XGBoost (10 puntos)

<p align="center">
  <img src="https://64.media.tumblr.com/14cc45f9610a6ee341a45fd0d68f4dde/20d11b36022bca7b-bf/s640x960/67ab1db12ff73a530f649ac455c000945d99c0d6.gif">
</p>

Un colega aficionado a la economía le *sopla* que la demanda guarda una relación inversa con el precio del producto. Motivado para impresionar al querido corpóreo, se propone hacer uso de esta información para mejorar su modelo realizando las siguientes tareas:

1. Vuelva a entrenar el `Pipeline` con `XGBRegressor`, pero esta vez forzando una relación monótona negativa entre el precio y la cantidad. Para aplicar esta restricción apóyese en la siguiente <a href = https://xgboost.readthedocs.io/en/stable/tutorials/monotonic.html>documentación</a>. [6 puntos]

>Hint 1: Para implementar el constraint se le sugiere hacerlo especificando el nombre de la variable. De ser así, probablemente le sea útil **mantener el formato de pandas** antes del step de entrenamiento.

>Hint 2: Puede obtener el nombre de las columnas en el paso anterior al modelo regresor mediante el método `.get_feature_names_out()`

2. Luego, vuelva a reportar el `MAE` sobre el conjunto de validación. [1 puntos]

3. ¿Cómo cambia el error al incluir esta relación? ¿Tenía razón su amigo? [2 puntos]

4. Guarde su modelo en un archivo .pkl [1 punto]

**1. Reentrenamiento `Pipeline` con `XGBRegressor` y relaciones**

In [231]:
# Obtenemos el nombre de las columnas posteriores a la transformación
X_train_copy = X_train.copy()
X_train_copy = date_transformer.fit_transform(X_train_copy)
col_transformer.fit(X_train_copy).set_output(transform='pandas')
feature_names = col_transformer.get_feature_names_out()
display(feature_names)

array(['num__lat', 'num__long', 'num__pop', 'num__capacity', 'num__price',
       'cat__city_Athens', 'cat__city_Irakleion', 'cat__city_Larisa',
       'cat__city_Patra', 'cat__city_Thessaloniki', 'cat__shop_shop_1',
       'cat__shop_shop_2', 'cat__shop_shop_3', 'cat__shop_shop_4',
       'cat__shop_shop_5', 'cat__shop_shop_6', 'cat__brand_adult-cola',
       'cat__brand_gazoza', 'cat__brand_kinder-cola',
       'cat__brand_lemon-boost', 'cat__brand_orange-power',
       'cat__container_can', 'cat__container_glass',
       'cat__container_plastic', 'cat__day_28', 'cat__day_29',
       'cat__day_30', 'cat__day_31', 'cat__month_1', 'cat__month_2',
       'cat__month_3', 'cat__month_4', 'cat__month_5', 'cat__month_6',
       'cat__month_7', 'cat__month_8', 'cat__month_9', 'cat__month_10',
       'cat__month_11', 'cat__month_12', 'cat__year_2012',
       'cat__year_2013', 'cat__year_2014', 'cat__year_2015',
       'cat__year_2016', 'cat__year_2017', 'cat__year_2018'], dtype=object)

In [232]:
# Restricciones en las variables
monotone_constraints = {name: -1 if 'num__price' in name else 0 for name in feature_names}

xgbr_const_model = XGBRegressor(monotone_constraints=monotone_constraints)

xgbr_pipe_with_constraints = Pipeline([
    ('date', date_transformer),
    ('col', col_transformer),
    ('model', xgbr_const_model)
], verbose=True)

xgbr_pipe_with_constraints.fit(X_train, y_train)

train_pred = xgbr_pipe_with_constraints.predict(X_train)
test_pred = xgbr_pipe_with_constraints.predict(X_test)

[Pipeline] .............. (step 1 of 3) Processing date, total=   0.0s
[Pipeline] ............... (step 2 of 3) Processing col, total=   0.0s
[Pipeline] ............. (step 3 of 3) Processing model, total=   0.4s


**2. Revisión de `MAE`**

In [233]:
mae_train = mean_absolute_error(y_train, train_pred)
mae_test = mean_absolute_error(y_test, test_pred)

print(f'MAE Train: {mae_train:.2f}')
print(f'MAE Test: {mae_test:.2f}')

MAE Train: 1263.35
MAE Test: 2703.58


**3. ¿Cambios en el error?**

**4. Guardado del modelo**

In [234]:
# Guardar modelo entrenado
joblib.dump(xgbr_pipe_with_constraints, 'xgbr_pipe_with_constraints.pkl')

['xgbr_pipe_with_constraints.pkl']

## 1.3 Optimización de Hiperparámetros con Optuna (20 puntos)

<p align="center">
  <img src="https://media.tenor.com/fmNdyGN4z5kAAAAi/hacking-lucy.gif">
</p>

Luego de presentarle sus resultados, Fiu le pregunta si es posible mejorar *aun más* su modelo. En particular, le comenta de la optimización de hiperparámetros con metodologías bayesianas a través del paquete `optuna`. Como usted es un aficionado al entrenamiento de modelos de ML, se propone implementar la descabellada idea de su jefe.

A partir de la mejor configuración obtenida en la sección anterior, utilice `optuna` para optimizar sus hiperparámetros. En particular, se pide que su optimización considere lo siguiente:

- Fijar una semilla en las instancias necesarias para garantizar la reproducibilidad de resultados
- Utilice `TPESampler` como método de muestreo
- De `XGBRegressor`, optimice los siguientes hiperparámetros:
    - `learning_rate` buscando valores flotantes en el rango (0.001, 0.1)
    - `n_estimators` buscando valores enteros en el rango (50, 1000)
    - `max_depth` buscando valores enteros en el rango (3, 10)
    - `max_leaves` buscando valores enteros en el rango (0, 100)
    - `min_child_weight` buscando valores enteros en el rango (1, 5)
    - `reg_alpha` buscando valores flotantes en el rango (0, 1)
    - `reg_lambda` buscando valores flotantes en el rango (0, 1)
- De `OneHotEncoder`, optimice el hiperparámetro `min_frequency` buscando el mejor valor flotante en el rango (0.0, 1.0)

Para ello se pide los siguientes pasos:
1. Implemente una función `objective()` que permita minimizar el `MAE` en el conjunto de validación. Use el método `.set_user_attr()` para almacenar el mejor pipeline entrenado. [10 puntos]
2. Fije el tiempo de entrenamiento a 5 minutos. [1 punto]
3. Optimizar el modelo y reportar el número de *trials*, el `MAE` y los mejores hiperparámetros encontrados. ¿Cómo cambian sus resultados con respecto a la sección anterior? ¿A qué se puede deber esto? [3 puntos]
4. Explique cada hiperparámetro y su rol en el modelo. ¿Hacen sentido los rangos de optimización indicados? [5 puntos]
5. Guardar su modelo en un archivo .pkl [1 punto]

In [ ]:
import optuna
from optuna.samplers import TPESampler
optuna.logging.set_verbosity(optuna.logging.WARNING)

def objective(trial):
    '''
    Función objetivo para optimizar el modelo XGBRegressor, buscando la mejor
    combinación de hiperparámetros para minimizar el error absoluto medio.
    Se usará TPESampler como método de muestreo.

    Args
        trial: instancia de la clase Trial

    Returns
        mae: error absoluto medio
    '''
    # Semilla para reproducibilidad
    np.random.seed(30)

    # Definición de los hiperparámetros a optimizar para xgboost
    xgb_params = {
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.1),
        'n_estimators': trial.suggest_int('n_estimators', 50, 1000),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'max_leaves': trial.suggest_int('max_leaves', 0, 100),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 5),
        'reg_alpha': trial.suggest_float('reg_alpha', 0, 1),
        'reg_lambda': trial.suggest_float('reg_lambda', 0, 1)
    }

    # Definición de los hiperparámetros a optimizar para OneHotEncoder
    cat_encoder_params = {
        'min_frequency': trial.suggest_float('min_frequency', 0.0, 1.0)
    }

    # Cargar el modelo
    xgb_model = joblib.load('xgbr_pipe.pkl')

    


## 4. Optimización de Hiperparámetros con Optuna y Prunners (17 puntos)

<p align="center">
  <img src="https://i.pinimg.com/originals/90/16/f9/9016f919c2259f3d0e8fe465049638a7.gif">
</p>

Después de optimizar el rendimiento de su modelo varias veces, Fiu le pregunta si no es posible optimizar el entrenamiento del modelo en sí mismo. Después de leer un par de post de personas de dudosa reputación en la *deepweb*, usted llega a la conclusión que puede cumplir este objetivo mediante la implementación de **Prunning**.

Vuelva a optimizar los mismos hiperparámetros que la sección pasada, pero esta vez utilizando **Prunning** en la optimización. En particular, usted debe:

- Responder: ¿Qué es prunning? ¿De qué forma debería impactar en el entrenamiento? [2 puntos]
- Redefinir la función `objective()` utilizando `optuna.integration.XGBoostPruningCallback` como método de **Prunning** [10 puntos]
- Fijar nuevamente el tiempo de entrenamiento a 5 minutos [1 punto]
- Reportar el número de *trials*, el `MAE` y los mejores hiperparámetros encontrados. ¿Cómo cambian sus resultados con respecto a la sección anterior? ¿A qué se puede deber esto? [3 puntos]
- Guardar su modelo en un archivo .pkl [1 punto]

Nota: Si quieren silenciar los prints obtenidos en el prunning, pueden hacerlo mediante el siguiente comando:

```
optuna.logging.set_verbosity(optuna.logging.WARNING)
```

De implementar la opción anterior, pueden especificar `show_progress_bar = True` en el método `optimize` para *más sabor*.

Hint: Si quieren especificar parámetros del método .fit() del modelo a través del pipeline, pueden hacerlo por medio de la siguiente sintaxis: `pipeline.fit(stepmodelo__parametro = valor)`

Hint2: Este <a href = https://stackoverflow.com/questions/40329576/sklearn-pass-fit-parameters-to-xgboost-in-pipeline>enlace</a> les puede ser de ayuda en su implementación

In [ ]:
#!pip install optuna-integration[xgboost]

In [ ]:
# Inserte su código acá

## 5. Visualizaciones (5 puntos)

<p align="center">
  <img src="https://media.tenor.com/F-LgB1xTebEAAAAd/look-at-this-graph-nickelback.gif">
</p>


Satisfecho con su trabajo, Fiu le pregunta si es posible generar visualizaciones que permitan entender el entrenamiento de su modelo.

A partir del siguiente <a href = https://optuna.readthedocs.io/en/stable/tutorial/10_key_features/005_visualization.html#visualization>enlace</a>, genere las siguientes visualizaciones:

1. Gráfico de historial de optimización [1 punto]
2. Gráfico de coordenadas paralelas [1 punto]
3. Gráfico de importancia de hiperparámetros [1 punto]

Comente sus resultados:

4. ¿Desde qué *trial* se empiezan a observar mejoras notables en sus resultados? [0.5 puntos]
5. ¿Qué tendencias puede observar a partir del gráfico de coordenadas paralelas? [1 punto]
6. ¿Cuáles son los hiperparámetros con mayor importancia para la optimización de su modelo? [0.5 puntos]

In [ ]:
# Inserte su código acá

## 6. Síntesis de resultados (3 puntos)

Finalmente:

1. Genere una tabla resumen del MAE en el conjunto de validación obtenido en los 5 modelos entrenados desde Baseline hasta XGBoost con Constraints, Optuna y Prunning. [1 punto]
2. Compare los resultados de la tabla y responda, ¿qué modelo obtiene el mejor rendimiento? [0.5 puntos]
3. Cargue el mejor modelo, prediga sobre el conjunto de **test** y reporte su MAE. [0.5 puntos]
4. ¿Existen diferencias con respecto a las métricas obtenidas en el conjunto de validación? ¿Porqué puede ocurrir esto? [1 punto]

In [ ]:
# Inserte su código acá

# Conclusión
Eso ha sido todo para el lab de hoy, recuerden que el laboratorio tiene un plazo de entrega de una semana. Cualquier duda del laboratorio, no duden en contactarnos por mail o U-cursos.

<p align="center">
  <img src="https://media.tenor.com/8CT1AXElF_cAAAAC/gojo-satoru.gif">
</p>

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=87110296-876e-426f-b91d-aaf681223468' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>